In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [2]:
col_names = names=['user_id', 'movie_id', 'ratings', 'timestamp']
df = pd.read_csv("../Data/MovieLens/ml-100k/u.data", sep='\t', names=col_names, engine='python')
df.drop('timestamp', axis=1, inplace=True)

In [3]:
n_users = df["user_id"].max()
n_items = df["movie_id"].max()

In [4]:
rating_mat = np.zeros((n_users, n_items))
for row in df.itertuples():
    rating_mat[row[1]-1, row[2]-1] = row[3]
rating_mat = np.array(np.vectorize(lambda x: 0 if x==0 else 1)(rating_mat))

In [5]:
movie_rated = np.zeros((len(df), n_items))
for i in range(len(df)):
    movie_rated[i] = rating_mat[df["user_id"][i]-1]
movie_rated = movie_rated / movie_rated.sum(axis = 1)[:, np.newaxis]
movie_rated_df = pd.DataFrame(movie_rated)

In [6]:
user_item_onehot = pd.get_dummies(df.loc[:,['user_id','movie_id']], columns=['user_id', 'movie_id'])

In [7]:
X =pd.concat([user_item_onehot, movie_rated_df],axis=1)
y = df.loc[:,'ratings']

In [8]:
X_train, X_test = train_test_split(X, test_size=0.2, random_state=22)
y_train, y_test = train_test_split(y, test_size=0.2, random_state=22)

In [9]:
class PandasDataset(Dataset):
    def __init__(self, X, y):
        """
        Args : X = x
                y= y
        """
        super(PandasDataset, self).__init__()
        self.X = X
        self.y = y
        
        self.X_value, self.y_value = self.X.values, self.y.values 
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return{
                'X' : torch.from_numpy(self.X_value)[idx],
                'y' : torch.from_numpy(self.y_value)[idx]
        }
        

In [10]:
batch_size = 1000

train_dataset = PandasDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle = True)

In [11]:
# test data
test_X = torch.FloatTensor(X_test.values)
test_y = torch.FloatTensor(y_test.values)

In [12]:
k = 20
n_features = X_train.shape[1]

In [13]:
class FM(nn.Module):
    def __init__(self):
        super(FM, self).__init__()
        
        self.w_0 = nn.Parameter(nn.init.normal_(torch.zeros((1, ))), requires_grad=True) # scalar
        self.w_i = nn.Parameter(nn.init.normal_(torch.zeros((1, n_features)), std=1.0/n_features), requires_grad = True) # 1 x 13698
        self.V = nn.Parameter(nn.init.normal_(torch.zeros((n_features, k)), std=1.0/k), requires_grad = True) # 13698 x 40
        
    def forward(self, X):
        
        bias = self.w_0 + torch.mm(X, self.w_i.T) # batch x 1
        
        term1 = torch.sum(torch.matmul(X, self.V), dim=1)**2 # 1 x batch
        term2 = torch.sum(torch.matmul(X, self.V)**2, dim=1) # 1 x batch
        interaction = 0.5*(term1-term2) # 1 x batch
        
        output = bias + interaction.view(-1,1)
        
        return output
        

In [14]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
device = torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [15]:
model = FM()
model.to(device)

FM()

In [16]:
epochs = 300
loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [17]:
for epoch in range(epochs):
    total_loss = 0
    model.train()
    for i, data in enumerate(train_loader):
        # forward pass
        pred = model(data['X'].float().to(device))
        loss = torch.sqrt(loss_fn(pred.squeeze(), data['y'].float().to(device)))
        
        # initialize
        optimizer.zero_grad()
        
        # backward pass
        loss.backward()
        
        # update
        optimizer.step()
        
        total_loss += loss.item()
        
    train_loss = total_loss / len(train_loader)
    
    # evaluation
    model.eval()
    
    pred = model(test_X.to(device))
    test_loss = torch.sqrt(loss_fn(pred.squeeze(), test_y.to(device)))
    
    if (epoch+1) % 10 == 0:
        print("epoch : {}, train_loss : {}, test_loss : {}".format(epoch+1, train_loss, test_loss))

epoch : 10, train_loss : 2.7861006259918213, test_loss : 2.757370948791504
epoch : 20, train_loss : 2.1510207831859587, test_loss : 2.126890182495117
epoch : 30, train_loss : 1.638580073416233, test_loss : 1.622286081314087
epoch : 40, train_loss : 1.316809518635273, test_loss : 1.3103936910629272
epoch : 50, train_loss : 1.179527571797371, test_loss : 1.1795697212219238
epoch : 60, train_loss : 1.1375068709254266, test_loss : 1.1396342515945435
epoch : 70, train_loss : 1.1259477257728576, test_loss : 1.1285741329193115
epoch : 80, train_loss : 1.1223115161061288, test_loss : 1.1250793933868408
epoch : 90, train_loss : 1.1206651270389556, test_loss : 1.1234776973724365
epoch : 100, train_loss : 1.119482970237732, test_loss : 1.1223644018173218
epoch : 110, train_loss : 1.118449442088604, test_loss : 1.1213877201080322
epoch : 120, train_loss : 1.1174312710762024, test_loss : 1.1204578876495361
epoch : 130, train_loss : 1.1164221450686456, test_loss : 1.1195497512817383
epoch : 140, tra

In [18]:
# 1.11 / 1.11